In [1]:
import numpy as np
from numpy.polynomial import polynomial as pl

# 1

### 1.1 The solutions of a polynomial congruence with prime modulus can always be found by testing, that is, by substituting each of the p numbers into the polynomial congruence to see if it does indeed satisfy the equation.

In [2]:
p1 = np.poly1d([2, 0, 0, 1, 1])

In [3]:
def solution_prime(polynom, mod_prime):
    #trying all the residue classes
    sol = []
    for i in range(mod_prime):
        if pl.polyval(i, polynom)%mod_prime==0:           
            sol.append(i)
    # print([f'{i}(mod{mod_prime})' for i in sol])
    return sol

In [4]:
solution_prime(p1, 7)

[4]

### the answer is 4 (mod 7)

## 1.2 Prime power modulus

![SNOWFALL](mod.png) 

### Let's compute prime decompositon of the modulo

In [5]:
def primeFactors(n):
    '''A function to get all prime factors of a given number n'''
    prime_list = []
    # Print the number of two's that divide n
    while n % 2 == 0:
        prime_list.append(2)
        n = n / 2
         
    # n must be odd at this point
    # so a skip of 2 ( i = i + 2) can be used
    for i in range(3,int(n**0.5)+1,2):
         
        # while i divides n , print i and divide n
        while n % i== 0:
            prime_list.append(i)
            n = n / i
             
    # Condition if n is a prime
    # number greater than 2
    if n > 2:
        prime_list.append(n)
    return {i:prime_list.count(i) for i in prime_list}

In [6]:
p2 = np.poly1d([1, 1, 0, 0, 0, 0, 0, 1])
mod = 343 

In [7]:
dict_of_factors = primeFactors(mod)
print(dict_of_factors)
print(f'The modulo is power 3 of 7')

{7: 3}
The modulo is power 3 of 7


In [8]:
def solution_prime_deg(polynom, prime, deg):
    '''polynom - np.poly1d polinom
    mod module of the congruency
    the function works only for mode - degree of a prime
    '''
    ### Step 1 - Solve for prime
    roots = solution_prime(polynom, prime)
    ### Step 2 solve for each degree of the prime as described above
    final_roots = f_roots(roots, 1, deg, polynom, prime)
    return final_roots

In [9]:
def f_roots(list_of_roots, i_deg, deg, polynom, prime):
    if i_deg==deg:
        return list_of_roots
    for root in list_of_roots:
        new_roots = find_roots(root, polynom, prime, i_deg)
        return f_roots(new_roots, i_deg+1, deg, polynom, prime)

In [10]:
def find_roots(root, polynom, prime, i_deg):
    '''function to find roots according to the formula above'''
    a0 = (pl.polyval(root, polynom)/(prime**i_deg))%prime
    a1 = pl.polyval(root, pl.polyder(polynom))%prime
    if a0==0 and a1==0:
        print(f'{prime} solutions, case not coded yet')
        return []
    roots_sub_ = solution_prime(np.poly1d([a0, a1]), prime)
    roots_sub = [root+r*prime**i_deg for r in roots_sub_]
    return roots_sub

In [11]:
solution_prime_deg(p2, 7, 3)

[115]

In [12]:
#check
pl.polyval(115, p2)%343==0

True

### The answer is 115 (mod 343)

# 2. By chinese Remainder Theorem, if mods of a system are pairwise coprime, the solution is unique and is obtained by a formula

![SNOWFALL](crt.png) 

In [13]:
def find_inverse_mod(elem, mod):
    '''if elem and mod are coprimes the inverse is unique and can be found by testing'''
    if gcd(elem, mod)==1:
        for i in range(1, mod):
            if elem*i%mod==1:
                return i

### The second equation can be replaced with x=1(mod2), then the modes are pairwise coprime. Let's verify it in code

In [14]:
def gcd(a, b): 
    '''gcd of numbers a and b'''  
    if (b == 0): 
        return a 
    else: 
        return gcd(b, a % b) 
  

def allCoprime(A):
    '''Function to check if all the pairs of the array are coprime with each other or not'''
    n = len(A)
    all_coprime = True
      
    for i in range(n):
        for j in range(i + 1, n):
              
            # Check if GCD of the pair is not equal to 1
            if gcd(A[i], A[j]) != 1:
   
                # All pairs are non-coprime
                all_coprime = False;
                break
      
    return all_coprime

In [15]:
def get_canon(elem_list, mod_list, rhs_list):
    m_list = [find_inverse_mod(elem, mod)*rhs%mod for elem, mod, rhs in zip(elem_list, mod_list, rhs_list)]
    return np.array(m_list)

In [16]:
elem_arr = np.array([2, 1, 4, 5])
mod_arr = np.array([5, 2, 7, 11])
rhs_array = np.array([1, 1, 1, 9]) #right hand side

In [17]:
def solve_system_lin_cong(elem_arr, mod_arr, rhs_array):
    ''' provides a solution to the system of linear congruencies, only the case of coprimes is considered'''
    # check if the modes are pairwise coprime, so we can use Chinese reminder theorem
    if allCoprime(mod_arr):
        # get canonical form of the system, meanig that each eq is x=rhs(mod_)
        rhs_canon_arr = get_canon(elem_arr, mod_arr, rhs_array)
        # Use the formula for Chinese reminder theorem
        M = mod_arr.prod()
        M_arr = M/mod_arr     
        b_arr = get_canon(M_arr, mod_arr, np.ones(4, dtype='int'))      
        X = (M_arr*b_arr*rhs_canon_arr).sum()
        return X

In [18]:
X = solve_system_lin_cong(elem_arr, mod_arr, rhs_array)
print(X)

3733.0


In [19]:
rhs_obt = X*elem_arr%mod_arr
rhs_obt

array([1., 1., 1., 9.])

In [20]:
(rhs_obt == rhs_array).all()

True

## The answer is 3733, the solution is correct

reference:
http://dspace.univ-msila.dz:8080/xmlui/bitstream/handle/123456789/16012/Amroune%20Abir.pdf?sequence=1&isAllowed=y

# 3. ?

# 4

In [21]:
def pad_polys(*polynomials):
    '''function to pad arrays with 0 in case they are of different degree'''
    L = max([len(i) for i in polynomials])
    args_out = [np.concatenate([p, np.zeros(L-len(p))]) for p in polynomials]
    return args_out

def add_poly(*polynomials):
    '''add polynomials'''
    poly_to_add = pad_polys(*polynomials)    
    args_out = np.concatenate([p.reshape(1,-1) for p in poly_to_add], 0)
    return args_out.sum(0)


def x_mult(p, deg):
    '''multiply polinomialby x'''
    return np.concatenate([np.zeros(deg), p])

In [22]:
def poly_prod_recursion(poly1, poly2):
    '''poly1, poly2 - np. arrays representing coeff of polynomials. 
    The first polynomial should be shorter
    '''
    ## base case
    deg1, deg2 = len(poly1), len(poly2)
    if deg1 == 1:
        return poly1[0]*poly2
    # Using the given formula    
    split_size1, split_size2 = deg1//2, deg2//2
    split10, split11 = poly1[:split_size1], x_mult(poly1[split_size1:], split_size1-1)
    split20, split21 = poly2[:split_size2], x_mult(poly2[split_size2:], split_size2-1)
    c0 = poly_prod_recursion(split10, split20)
    c2 = poly_prod_recursion(split11, split21)
    s1 = add_poly(split11,-split10)
    s2 = add_poly(split21, -split20)
    c1 = add_poly(c0, c2, -poly_prod_recursion(s1, s2))
    return add_poly(x_mult(c2, 2), x_mult(c1, 1), c0)

In [23]:
def poly_prod(poly1, poly2):
    '''Returns np.array representing multiplication of two polynomials
    '''
    # chose shorter polynom to be first in the function call is of the same or smaller degree than poly2   
    L = len(poly1)
    if len(poly2)<L:
        return poly_prod_recursion(poly2, poly1)
    else:
        return poly_prod_recursion(poly1, poly2)

### Compare to the numpy implementation to make sure it works correctly

In [24]:
(poly_prod(p1.c, p2.c) == pl.polymul(p1, p2)).all()

True

In [25]:
(poly_prod(p1.c, p1.c) == pl.polymul(p1, p1)).all()

True

In [26]:
(poly_prod(p2.c, p2.c) == pl.polymul(p2, p2)).all()

True